In [93]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [94]:
df = pd.read_csv('rideshare_kaggle.csv')

In [1]:
from scipy.stats import spearmanr

In [ ]:
target = 'price'

In [98]:
numeric_cols = df.select_dtypes(include='number').columns.drop(target)

In [99]:
# Calcula a correlação de Spearman entre cada coluna e o target
corr_results = {
    col: spearmanr(df[col], df[target], nan_policy='omit').correlation
    for col in numeric_cols
}

In [100]:
# Transforma em DataFrame
corr_df = pd.DataFrame.from_dict(corr_results, orient='index', columns=['corr_coef'])
corr_df = corr_df.sort_values(by='corr_coef', ascending=False)
corr_df.where(corr_df >= 0).head(6)

,corr_coef
distance,0.333871
surge_multiplier,0.165611
latitude,0.002037
apparentTemperatureLow,0.001378
pressure,0.001282
temperatureHigh,0.000861


In [101]:
from scipy.stats import chi2_contingency

In [102]:
cat_cols = df.select_dtypes(include=['object', 'string']).columns
cat_cols

Index(['id', 'datetime', 'timezone', 'source', 'destination', 'cab_type',
       'product_id', 'name', 'short_summary', 'long_summary', 'icon'],
      dtype='object')

In [103]:
df = df.dropna(subset=cat_cols.tolist() + [target])

In [104]:
# Armazenar resultados
chi2_results = []

for col in cat_cols:
    # Tabela de contingência
    contingency_table = pd.crosstab(df[col], df[target])
    
    # Verifica se a tabela de contingência é válida
    if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:
        # Aplica o teste qui-quadrado
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        # Salva resultado
        chi2_results.append({'feature': col, 'p_value': p, 'chi2_stat': chi2})
    else:
        # Ignora colunas com tabelas inválidas
        chi2_results.append({'feature': col, 'p_value': None, 'chi2_stat': None})

In [105]:
# Transforma em DataFrame e ordena pelo p-valor
chi2_df = pd.DataFrame(chi2_results).sort_values(by='p_value')
chi2_df.head(8)

,feature,p_value,chi2_stat
3,source,0.000000e+00,9.246860e+04
4,destination,0.000000e+00,9.066977e+04
5,cab_type,0.000000e+00,3.890842e+05
6,product_id,0.000000e+00,2.331125e+06
7,name,0.000000e+00,2.331125e+06
1,datetime,1.272108e-40,4.617290e+06
9,long_summary,1.435545e-01,1.517598e+03
8,short_summary,4.005883e-01,1.179543e+03


In [106]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

colunas que iremos usar para o modelo.

'distance', 'surge_multiplier', 'latitude', 'apparentTemperatureLow', 'pressure', 'temperatureHigh', 'source', 'destination', 'cab_type', 'name', 'long_summary', 'short_summary'

In [ ]:
df = df[['distance', 'surge_multiplier', 'latitude',
         'apparentTemperatureLow', 'pressure', 'temperatureHigh',
         'source', 'destination', 'cab_type',
         'name', 'long_summary', 'short_summary', 'price']]

Target-Guided Encoding com Mediana (Target Median Encoding)

Essa codificação com median() é robusta a outliers, ideal para preços.

In [ ]:
# Codificação das variáveis categóricas com base na mediana do target
for col in df.select_dtypes(include='object').columns:
    median_map = df.groupby(col)['price'].median()
    df[col] = df[col].map(median_map)

In [109]:
X = df.drop(columns=['price'])  # substitua 'preco' pelo seu target
y = df['price']

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Padronização dos dados (mean = 0, std = 1)

In [111]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [114]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [115]:
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=42, n_jobs=-1)

In [116]:
rf.fit(X=X_train_scaled, y=y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [117]:
y_pred = rf.predict(X_test_scaled)

In [118]:
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

MSE: 3.3030751568746526
R²: 0.962112191494016


In [119]:
df.price.describe()

count    637976.000000
mean         16.545125
std           9.324359
min           2.500000
25%           9.000000
50%          13.500000
75%          22.500000
max          97.500000
Name: price, dtype: float64

# 📊 Análise de Resultados — Modelo de Previsão de Preços (Uber)

Este documento apresenta a interpretação dos principais resultados obtidos com o modelo de machine learning aplicado à previsão dinâmica de preços de corridas do Uber. Os valores de **preço estão em dólar (USD)**.

---

## 🎯 Resumo das Métricas

| Métrica  | Valor         | Interpretação                                                  |
|----------|---------------|-----------------------------------------------------------------|
| R²       | 0.96          | O modelo explica 96% da variabilidade dos preços. Excelente.   |
| MSE      | 3.3           | Erro quadrático médio. Penaliza mais erros grandes.            |
| RMSE     | ≈ 1.82        | Erro médio absoluto (interpretação direta em USD).             |


---

## 📌 Interpretação do RMSE

O RMSE (Root Mean Squared Error) de **~1.82 USD** indica que, **em média, o modelo erra cerca de $1.82 por previsão**.

Comparando com as estatísticas do preço:

- Média do preço: **$16.55**
    
- Erro médio (RMSE): **$1.82**
    
- Proporção do erro: **~11%** do valor médio
    
- O RMSE está **bem abaixo do desvio padrão ($9.32)**, o que é um ótimo sinal de desempenho.
    

---

## ✅ Conclusão

Com base nas métricas:

- O modelo está **altamente ajustado** aos dados, com **baixo erro médio** e **alta capacidade explicativa**.
    
- O erro de previsão médio de **$1.82** é aceitável, considerando que o preço médio das corridas é de aproximadamente **$16.55**.
    
- O **MSE de 3.3** é coerente e reforça que o modelo **não comete erros extremos com frequência**.
    

Esse desempenho indica que o modelo está **bem calibrado e pronto para uso prático**, podendo ser utilizado para prever preços em contextos de precificação dinâmica com boa precisão.

testar: MAE, MAPE